<a href="https://colab.research.google.com/github/Joetech-hub/my_repo/blob/main/stageCQuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
import numpy as np


In [32]:
df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn.csv')
pd.set_option('display.max_column',40)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [34]:
# convert object colum to numerical
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')


In [35]:
# Fill missing values with 0
df['TotalCharges'] = df['TotalCharges'].fillna(0)


In [36]:
# Encode the target vector
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df.Churn = encoder.fit_transform(df.Churn)


In [37]:
# Declare variables for feature matrix and target vector
x = df.drop(columns=['Churn','customerID'])
y = df['Churn']

In [38]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [39]:
# Scale numerical features and convert output back to DataFrame
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_train = scaler.fit_transform(X_train[['tenure','MonthlyCharges','TotalCharges']])
scaled_test = scaler.transform(X_test[['tenure','MonthlyCharges','TotalCharges']])

scaled_train_df = pd.DataFrame(scaled_train, columns=X_train[['tenure','MonthlyCharges','TotalCharges']].columns)
scaled_test_df = pd.DataFrame(scaled_test, columns=X_test[['tenure','MonthlyCharges','TotalCharges']].columns)

In [40]:
# Encode numerical features and convert output back to DataFrame with the column names
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
encoded_X_train = encoder.fit_transform(X_train.drop(columns=['tenure','MonthlyCharges','TotalCharges']))
encoded_X_test = encoder.transform(X_test.drop(columns=['tenure','MonthlyCharges','TotalCharges']))

encoded_df_train = pd.DataFrame(encoded_X_train, columns=encoder.get_feature_names_out(X_train.drop(columns=['tenure','MonthlyCharges','TotalCharges']).columns))
encoded_df_test = pd.DataFrame(encoded_X_test, columns=encoder.get_feature_names_out(X_test.drop(columns=['tenure','MonthlyCharges','TotalCharges']).columns))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [41]:
# Combine scaled and encoded features into train and test set
X_train = pd.concat([scaled_train_df, encoded_df_train], axis=1)
X_test = pd.concat([scaled_test_df, encoded_df_test], axis=1)

In [42]:
# import libraries for model training and test evaluation
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [43]:
# Random Forest Classifier
rf = RandomForestClassifier(random_state=1)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_accu = accuracy_score(y_test, rf_pred)

rf_accu

0.7913413768630234

In [44]:
# Extra Trees Classifier
ex_tree = ExtraTreesClassifier(random_state=1)

ex_tree.fit(X_train, y_train)
ex_tree_pred = ex_tree.predict(X_test)
ex_tree_acc = accuracy_score(y_test, ex_tree_pred)

ex_tree_acc

0.7672107877927609

In [45]:
# XGB Classifier
xgb = XGBClassifier(random_state=1)

xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_accu = accuracy_score(y_test, xgb_pred)

xgb_accu

0.7934705464868701

In [63]:
# LGBM Classifier
lgbm = LGBMClassifier(random_state=1)

lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_test)
lgbm_accu = accuracy_score(y_test, lgbm_pred)

lgbm_accu

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1521, number of negative: 4113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000898 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 670
[LightGBM] [Info] Number of data points in the train set: 5634, number of used features: 46
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.269968 -> initscore=-0.994785
[LightGBM] [Info] Start training from score -0.994785


0.8034066713981547

In [66]:

from sklearn.tree import ExtraTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

ex_tree = ExtraTreesClassifier(random_state=1)

random_search = RandomizedSearchCV(ex_tree, hyperparameter_grid, n_iter=10, scoring='accuracy',cv=5, n_jobs=-1, verbose=1, random_state=1)
# Fit the RandomizedSearchCV to X_train and y_train
random_search.fit(X_train, y_train)

# Get the best hyperparameters
best_parameters = random_search.best_params_
best_parameters

Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'n_estimators': 1000,
 'min_samples_split': 9,
 'min_samples_leaf': 8,
 'max_features': 'sqrt'}